# Personal Investment Analysis

In [1]:
using PyPlot, DataFrames, NPFinancial, Dates, KeywordSearch, Random
using StatsPlots, Formatting, Dates, PrettyTables, XLSX, PlotThemes, MarketData
include("PersonalFinance.jl")
ht("<H3>Personal Investment Analysis<H3>")
ht("Report Date: $(today())")

Personal Investment Analysis

Report Date: 2023-06-25

In [2]:
dataDir="../data"
invSheet=joinpath(dataDir,"Investments.xlsx")
ht("Investment input: $invSheet")
invTab="Investments"
transSheet=joinpath(dataDir, "Transactions.xlsx")
ht("TransactionInput input: $transSheet")
inv=readTab(invSheet, invTab)
stockList=select(inv, [:Code, :Type]) 
stockList=filter(:Type => t -> t == "Stock", stockList)
stockList=stockList[!, :Code]
holdings=Symbol.(setdiff(inv.Code, ["House", "FidelityHP"]))
trans=readTab(transSheet, 1);
#formatters = (v, i, j) -> (j in [6] ) ? format(round(Int, v), commas=true,parens=true) : v
#pretty_table(inv; backend = Val(:html), nosubheader=true)

Investment input: ../data/Investments.xlsx

TransactionInput input: ../data/Transactions.xlsx

Row,Date,Account,Description,Category,Tags,Amount
,String,String,String,String,Any,Int64
1,2021-07-16,Brokeragelink - Ending in 9921,You Bought Toyota Motor Adr Rep 2 Ord,Securities Trades,missing,-3578


## Historical Analysis 

In [3]:
#Find sold investments
sold=  filter(:Amount => t -> t<(0.0), trans)
bought=filter(:Amount => t -> t>(0.0), trans)
#profit=outerjoin(sold, bought, on=:InvestmentID, makeunique=true)
pretty_table(bought; backend = Val(:html), nosubheader=true)
#profit=select(profit, [1,3,4,5,8])
#rename!(profit, ["ID", "AmountSold", "SharesSold", "DateSold", "Basis"])

Date,Account,Description,Category,Tags,Amount


In [4]:
opts=PolyOpts("SgTZRxROKKu6NrZ9SMSc60myLze5jgOa", DataFrame)
function getstock_daily(opts, holding, day)
  try 
#    stocks_daily_open_close(opts, holding, day)
     stocks_previous_close(opts,holding)
  catch 
    try 
#      stocks_daily_open_close(opts, holding, day)
    sleep(5)
    stocks_previous_close(opts,holding)
    catch
      display("Unable to get price for $holding on $day")
      missing
    end
  end
end
stocklist_daily(opts, holdings, day=today())=vcat(map(h-> getstock_daily(opts, h, day), holdings)...);


UndefVarError: UndefVarError: `PolyOpts` not defined

In [ ]:
function lastBusinessDay(d)
  dow=Dates.dayofweek(d)
  if dow in [2,3,4,5,6] #T-Sa
    offset=1
  elseif dow == 1 #M
    offset=3
  else
    offset=2 #Su
  end
  d-Dates.Day(offset)
end
function getHLOC(ticker, date)
  startDate=lastBusinessDay(date)
  try 
    df=DataFrame(yahoo(ticker, YahooOpt(period1=startDate, period2=startDate)))
  catch
    display("Unable to find ticker $ticker for $(string(startDate))")
    return missing
  end
  df=hcat(df, [ticker])
  DataFrames.rename!(df, 8 => "Ticker")
end

In [ ]:
date=DateTime(2021,8,20)
yahoo("MRNA", YahooOpt(period1=date))

In [ ]:
stocksYesterday=vcat(map(x -> getHLOC(x, today()), stockList)...)
pretty_table(stocksYesterday; backend = Val(:html), nosubheader=true)

In [ ]:
using KeywordSearch, Random
d1="Dividend Received As Of 07/31/2021 Jpmorgan Chase & Co"
s2="JPMorgan chase"
fuzzy_query=FuzzyQuery(s2)
m=match(fuzzy_query, d1)
display(m)
explain(m)
